#MSE - OLS,Ridge,Lasso

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files
import numpy as np
from scipy import stats
import seaborn as sns
import pandas as pd
import io
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 19.9MB/s 

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
id='1bcu2TSWXPFedsnmQV2BppNpnMU6YC6Qv' # The id File
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('FINAL_MASTER.csv')  
df = pd.read_csv('FINAL_MASTER.csv')


In [0]:
df.head()

,artist,date,weekday,promoter,SG_venue,venue_city,venue_state,TM_max,SG_max_price,TM_min,...,daystoconcert,segment,genre,subGenre,Capture_date,SG_listing_count,SG_venue_score,max_pop,max_follow,SG_artists_score
0,Lake Street Dive,1/15/2019,Tuesday,PROMOTED BY VENUE,Ponte Vedra Concert Hall,Ponte Vedra Beach,FL,34.5,36.0,34.5,...,0,Music,Rock,Alternative Rock,1/15/2019,3,0.466979,64,191086,0.56
1,Mineral,1/16/2019,Wednesday,LIVE NATION MUSIC,Theatre of Living Arts,Philadelphia,PA,25.0,109.0,25.0,...,1,Music,Rock,Alternative Rock,1/15/2019,35,0.503499,41,19707,0.39
2,Mineral,1/16/2019,Wednesday,LIVE NATION MUSIC,Theatre of Living Arts,Philadelphia,PA,25.0,68.0,25.0,...,0,Music,Rock,Alternative Rock,1/16/2019,5,0.503499,41,19746,0.39
3,Dylan Scott,1/17/2019,Thursday,LIVE NATION MUSIC,Gramercy Theatre,New York,NY,50.5,286.0,30.5,...,2,Music,Country,Country,1/15/2019,14,0.515187,66,372825,0.49
4,Vertical Horizon,1/17/2019,Thursday,PROMOTED BY VENUE,Highline Ballroom,New York,NY,75.0,217.0,35.0,...,2,Music,Rock,Alternative Rock,1/15/2019,10,0.506355,56,184769,0.42


#Data cleaning

In [0]:
##Removing rows accordingly to our data cleaning process
#Remove max popularity < 10
#Remove max followers <10
#Remove data points with average price > 400 (IQR 0.75 - 0.25 + Mean)
#Drop any lines with NA, None
#Remove venue score 0 and artist score 0

df = df[df['SG_average_price'] > 0]

df=df[df['SG_average_price'] <= 400]

df=df[df['max_pop'] >=10]

df=df[df['max_follow'] >=10]

df=df[df['SG_venue_score'] >=0.1]

df=df[df['SG_artists_score'] >=0.1]



In [0]:
#drop NAs
df.isna().sum()
df= df.dropna()
df.isna().sum()


artist              0
date                0
weekday             0
promoter            0
SG_venue            0
venue_city          0
venue_state         0
TM_max              0
SG_max_price        0
TM_min              0
SG_min_price        0
SG_average_price    0
daystoconcert       0
segment             0
genre               0
subGenre            0
Capture_date        0
SG_listing_count    0
SG_venue_score      0
max_pop             0
max_follow          0
SG_artists_score    0
dtype: int64

In [0]:
df.shape

(11861, 22)

In [0]:
#Change to numeric

df["SG_artists_score"] = pd.to_numeric(df["SG_artists_score"])
df["SG_venue_score"] = pd.to_numeric(df["SG_venue_score"])
df["max_pop"] = pd.to_numeric(df["max_pop"])
df["max_follow"] = pd.to_numeric(df["max_follow"])
df["daystoconcert"] = pd.to_numeric(df["daystoconcert"])
df["SG_average_price"] = pd.to_numeric(df["SG_average_price"])

#Change text object to String

df[["promoter", "genre","subGenre","venue_city","venue_state","SG_venue","weekday"]] = df[["promoter", "genre","subGenre","venue_city","venue_state", "SG_venue","weekday"]].astype(str) 


#Coerce string to categories
df["promoter"] = df["promoter"].astype('category')
df["genre"] = df["genre"].astype('category')
df["subGenre"] = df["subGenre"].astype('category')
df["venue_city"] = df["venue_city"].astype('category')
df["venue_state"] = df["venue_state"].astype('category')
df["weekday"] = df["weekday"].astype('category')
df["SG_venue"] = df["SG_venue"].astype('category')


In [0]:
df.dtypes

artist                object
date                  object
weekday             category
promoter            category
SG_venue            category
venue_city          category
venue_state         category
TM_max               float64
SG_max_price         float64
TM_min               float64
SG_min_price         float64
SG_average_price     float64
daystoconcert          int64
segment               object
genre               category
subGenre            category
Capture_date          object
SG_listing_count       int64
SG_venue_score       float64
max_pop                int64
max_follow             int64
SG_artists_score     float64
dtype: object

#OLS Model

In [0]:
#import libraries for modeling
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC


In [0]:
#Removing target column from ORIGINAL DF

##Data
#Output is Type pandas df
X = df[['SG_artists_score',
'venue_city',
'SG_venue_score',
'venue_state',
'max_pop',
'max_follow',
'daystoconcert',
        "genre",
       "weekday"]]

#Target
#Output is Type pandas df
Y = df[['SG_average_price']].astype(int)



In [0]:
X.head()

,SG_artists_score,venue_city,SG_venue_score,venue_state,max_pop,max_follow,daystoconcert,genre,weekday
0,0.56,Ponte Vedra Beach,0.466979,FL,64,191086,0,Rock,Tuesday
1,0.39,Philadelphia,0.503499,PA,41,19707,1,Rock,Wednesday
2,0.39,Philadelphia,0.503499,PA,41,19746,0,Rock,Wednesday
3,0.49,New York,0.515187,NY,66,372825,2,Country,Thursday
4,0.42,New York,0.506355,NY,56,184769,2,Rock,Thursday


In [0]:
#One Hot! 


dummies1 = pd.get_dummies(df['promoter'])
X= pd.concat([X,dummies1],axis=1)

dummies2 = pd.get_dummies(df['genre'])
X= pd.concat([X,dummies2],axis=1)

dummies3 = pd.get_dummies(df['subGenre'])
X= pd.concat([X,dummies3],axis=1)

dummies4 = pd.get_dummies(df['venue_city'])
X= pd.concat([X,dummies4],axis=1)

dummies5 = pd.get_dummies(df['venue_state'])
X= pd.concat([X,dummies5],axis=1)

dummies6 = pd.get_dummies(df['weekday'])
X= pd.concat([X,dummies6],axis=1)



In [0]:
#DROP COLUMNS THAT WE USE GET DUMMIES ON

X.drop(columns=['genre','venue_city','venue_state','weekday' ], inplace=True)


In [0]:
#Check if columns were removed
X.head()

,SG_artists_score,SG_venue_score,max_pop,max_follow,daystoconcert,191 TOURING,AEG - TICKETING,AEG LIVE,AL WASH,ANDREW HEWITT & BILL SILVA PRESENTS,...,PA,TN,TX,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0.56,0.466979,64,191086,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0.39,0.503499,41,19707,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,0.39,0.503499,41,19746,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,0.49,0.515187,66,372825,2,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0.42,0.506355,56,184769,2,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## Spliting data into test and train


In [0]:
##Import function for splitting arrays or matrices into random train and test subsets
from sklearn.model_selection import train_test_split

# Import StandardScaler
from sklearn.preprocessing import StandardScaler


# Split 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2,random_state=1)

In [0]:
print ("X.shape:", X.shape)
print ("Y.shape:", Y.shape)
print ("X_train.shape:", X_train.shape)
print ("X_test.shape:", X_test.shape)
print ("y_train.shape:", y_train.shape)
print ("y_test.shape:", y_test.shape)

X.shape: (11861, 230)
Y.shape: (11861, 1)
X_train.shape: (9488, 230)
X_test.shape: (2373, 230)
y_train.shape: (9488, 1)
y_test.shape: (2373, 1)


##Fitting, training and predicting models.

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)


In [0]:
# The coefficients
#print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 2599.57
Variance score: 0.53


#Lasoo and Ridge MSE


In [0]:
# Import linear models
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
# Create lasso and ridge objects
lasso = linear_model.Lasso()
ridge = linear_model.Ridge()
# Fit the models
lasso.fit(X, Y)
ridge.fit(X, Y)
# Print scores, MSE, and coefficients
print("lasso score:", lasso.score(X, Y))
print("ridge score:",ridge.score(X, Y))
print("lasso MSE:", mean_squared_error(Y, lasso.predict(X)))
print("ridge MSE:", mean_squared_error(Y, ridge.predict(X)))



lasso score: 0.3671883380980532
ridge score: 0.5669974653922331
lasso MSE: 3411.443019544524
ridge MSE: 2334.28611238463


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=3.1582e-17): result may not be accurate.
  overwrite_a=True).T


In [0]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=20, random_state=0)  
regressor.fit(X_train, y_train)  
y_pred = regressor.predict(X_test)  

from sklearn import metrics

print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


Mean Squared Error: 681.7239980641592
Root Mean Squared Error: 26.109844849484634


In [0]:
regressor1 = RandomForestRegressor(n_estimators=2000, random_state=0)  
regressor1.fit(X_train, y_train)  
y_pred1 = regressor1.predict(X_test)  

print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred1))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred1)))  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Mean Squared Error: 638.643409333896
Root Mean Squared Error: 25.271395080879408


In [0]:
regressor2 = RandomForestRegressor(n_estimators=200, random_state=0)  
regressor2.fit(X_train, y_train)  
y_pred2 = regressor2.predict(X_test)  

print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred2))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred2)))  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Mean Squared Error: 640.972792877963
Root Mean Squared Error: 25.31744048828718
